<a href="https://colab.research.google.com/github/phuonglinh81/Fast-conformer-NVIDIA-Vivos/blob/main/process_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tăng cường dữ liệu

In [ ]:
import os
import librosa
import numpy as np
from scipy.io.wavfile import write
from IPython.display import Audio, display
from audiomentations import Compose, TimeStretch
from scipy import signal
import soundfile as sf  # Thêm thư viện soundfile để lưu âm thanh

# Đường dẫn thư mục gốc và các thư mục cần thiết
# Đường dẫn đến thư mục data gốc
data_dir = '/content/drive/MyDrive/dataset_vivos/vivos/test/waves'

# Đường dẫn đến thư mục chứa data tăng cường
augment_dir = '/content/drive/MyDrive/dataset_vivos/vivos/augment/test_aug'
os.makedirs(augment_dir, exist_ok=True)

# Hàm lưu tệp âm thanh mới vào thư mục augment
def save_audio(y, sr, file_name):
    new_file_path = os.path.join(augment_dir, file_name)
    write(new_file_path, sr, (y * 32767).astype(np.int16))  # Chuẩn hóa dữ liệu để ghi dạng int16

# Thay đổi cao độ
def change_pitch(y, sr, n_steps=2):
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

# Thêm nhiễu ngẫu nhiên
def add_noise(y, noise_factor=0.005):
    return y + noise_factor * np.random.randn(len(y))

# Hàm thêm tiếng vang (reverberation)
def add_reverberation(y, sr, reverb_amount=0.4):
    ir_length = int(0.3 * sr)  # 300 ms
    ir = np.zeros(ir_length)
    ir[0] = 1
    ir[int(0.05 * sr)] = reverb_amount  # Thêm phản xạ tại 50 ms
    y_reverb = signal.convolve(y, ir)[:len(y)]  # Tính toán tiếng vang
    y_reverb = y_reverb / np.max(np.abs(y_reverb))  # Chuẩn hóa
    return y_reverb

# Lấy danh sách tệp âm thanh
audio_files = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.wav'):
            audio_files.append(os.path.join(root, file))

# Áp dụng các phép biến đổi theo tỉ lệ đã cho
for file_path in audio_files:
    y, sr = librosa.load(file_path, sr=None)
    file_name = os.path.basename(file_path).split('.')[0]

    # Chọn phép biến đổi ngẫu nhiên theo tỉ lệ
    transform_type = random.choices(
        ['pitch', 'noise', 'reverb', 'speed'],
        weights=[0.2, 0.3, 0.2, 0.1],  # Thêm weights cho reverberation
        k=1
    )[0]

    # Áp dụng phép biến đổi với mức độ ngẫu nhiên
    if transform_type == 'pitch':
        n_steps = random.randint(-2, 2)  # Thay đổi độ cao ngẫu nhiên
        y_augmented = change_pitch(y, sr, n_steps=n_steps)
        new_file_name = f"{file_name}_pitch.wav"
    elif transform_type == 'noise':
        noise_factor = random.uniform(0.001, 0.01)  # Thêm nhiễu ngẫu nhiên
        y_augmented = add_noise(y, noise_factor=noise_factor)
        new_file_name = f"{file_name}_noise.wav"
    elif transform_type == 'reverb':
        reverb_amount = random.uniform(0.1, 0.9)  # Lượng tiếng vang ngẫu nhiên
        y_augmented = add_reverberation(y, sr, reverb_amount=reverb_amount)
        new_file_name = f"{file_name}_reverb.wav"
    elif transform_type == 'speed':
        # Thay đổi tốc độ (time-stretch) sử dụng audiomentations
        augmentations = Compose([
            TimeStretch(min_rate=0.5, max_rate=1.5, p=1)  # Thay đổi tốc độ từ 0.5 đến 1.5
        ])
        y_augmented = augmentations(samples=y, sample_rate=sr)
        new_file_name = f"{file_name}_timestretch.wav"

    # Lưu tệp đã làm giàu
    save_audio(y_augmented, sr, new_file_name)
